In [12]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
os.chdir("..")

In [34]:
LOGS_ROOT_DIR = Path("lightning_logs")

In [35]:
logs_raw_paths = list(LOGS_ROOT_DIR.rglob("**/*.csv"))
logs_raw_paths

[PosixPath('lightning_logs/MNISTClassificationModule_MNISTProvider/raw_results.csv')]

In [36]:
logs_path = logs_raw_paths[0]
logs_path.parts[1]

'MNISTClassificationModule_MNISTProvider'

In [37]:
ACCURACY = "Classification" in logs_path.parts[1]
LOSS_METRIC = "CE" if ACCURACY else "MSE"


df = pd.read_csv(logs_path)
df.head()

,test_accuracy,test_loss,num_params,experiment,model,experiment_name,hidden_size,num_hidden,seed
0,0.8934,0.381226,18698,MNISTClassificationModule_MNISTProvider,MorpherNet,20240826_h128_identity_s37,128,identity,37
1,0.9535,0.174658,250122,MNISTClassificationModule_MNISTProvider,MorpherNet,20240826_h256_three_hidden_s53,256,three_hidden,53
2,0.9119,0.298071,20298,MNISTClassificationModule_MNISTProvider,MorpherNet,20240826_h64_three_hidden_s17,64,three_hidden,17
3,0.9193,0.262319,20298,MNISTClassificationModule_MNISTProvider,MorpherNet,20240826_h64_three_hidden_s19,64,three_hidden,19
4,0.9110,0.321513,52746,MNISTClassificationModule_MNISTProvider,MorpherNet,20240826_h256_identity_s19,256,identity,19


In [38]:
# Remap values of num_hidden
hidden_map = {
    "identity": 0,
    "one_hidden": 1,
    "three_hidden": 3
}
df["num_hidden"] = df["num_hidden"].apply(lambda x: hidden_map[x])

# Remap values of model
models_map = {
    "AutomorpherNet": "MorpherNet",
    "DepthwiseAutomorpherNet": "MorpherNetv2"
}
df["model"] = df["model"].apply(lambda x: models_map[x] if x in models_map else x)

if LOSS_METRIC == "MSE":
    df["test_loss"] = np.sqrt(df["test_loss"])
    LOSS_METRIC == "RMSE"

# Rename columns
cols_to_rename = ["test_loss"]
if ACCURACY:
    cols_to_rename.append("test_accuracy")
for col in cols_to_rename:
    df[f"{col}_mean"] = df[col].copy()
    df = df.rename(columns={col:f"{col}_stddev"})
df = df.rename(columns={"hidden_size": "hidden size", "num_hidden":"\# hidden", "num_params": "\# parameters"})

In [39]:
GROUPBY_COLUMNS = ["model", "\# hidden", "hidden size"]

aggregations_dict = {
            "test_loss_mean": np.mean, 
            "test_loss_stddev": np.std,
            "\# parameters": lambda x: x.values[0]
        }
if ACCURACY:
    aggregations_dict["test_accuracy_mean"] = np.mean
    aggregations_dict["test_accuracy_stddev"] = np.std

aggregated = df.groupby(GROUPBY_COLUMNS)\
    .agg(aggregations_dict)

/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_28856/516040104.py:13: FutureWarning: The provided callable <function mean at 0x1081743a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(aggregations_dict)
/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_28856/516040104.py:13: FutureWarning: The provided callable <function std at 0x1081744c0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(aggregations_dict)
/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_28856/516040104.py:13: FutureWarning: The provided callable <function mean at 0x1081743a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "

In [40]:
# val loss
aggregated[LOSS_METRIC] = aggregated["test_loss_mean"].apply(lambda x: f"{x:.4f}") \
    + " "\
    + aggregated["test_loss_stddev"].apply(lambda x: f"({x:.4f})")
aggregated = aggregated.drop(columns=["test_loss_mean", "test_loss_stddev"])

In [41]:
# accuracy
if ACCURACY:
    aggregated["accuracy"] = aggregated["test_accuracy_mean"].apply(lambda x: f"{x*100:.2f}\%") \
        + " "\
        + aggregated["test_accuracy_stddev"].apply(lambda x: f"({x*100:.2f}\%)")
    aggregated = aggregated.drop(columns=["test_accuracy_mean", "test_accuracy_stddev"])

In [42]:
cols_to_select = [LOSS_METRIC, "\# parameters"]
if ACCURACY:
    cols_to_select = ["accuracy"] + cols_to_select
aggregated = aggregated[cols_to_select]
aggregated

accuracy               CE  \
model        \# hidden hidden size                                      
BaselineNet  0         64           97.16\% (0.12\%)  0.0975 (0.0018)   
                       128          97.59\% (0.08\%)  0.0830 (0.0027)   
                       256          97.81\% (0.11\%)  0.0759 (0.0031)   
             1         64           97.17\% (0.12\%)  0.0993 (0.0027)   
                       128          97.48\% (0.16\%)  0.0869 (0.0056)   
                       256          97.78\% (0.22\%)  0.0824 (0.0064)   
             3         64           96.89\% (0.11\%)  0.1093 (0.0021)   
                       128          97.37\% (0.16\%)  0.0917 (0.0038)   
                       256          97.69\% (0.25\%)  0.0858 (0.0078)   
MorpherNet   0         64           87.68\% (0.21\%)  0.4359 (0.0031)   
                       128          89.43\% (0.14\%)  0.3743 (0.0045)   
                       256          90.93\% (0.10\%)  0.3229 (0.0020)   
             1         64           91.06\% (0.42\%)  0.3005 (0.0142)   
                       128          93.51\% (0.19\%)  0.2192 (0.0086)   
                       256          95.65\% (0.15\%)  0.1490 (0.0088)   
             3         64           91.61\% (0.47\%)  0.2732 (0.0182)   
                       128          94.87\% (0.17\%)  0.1782 (0.0061)   
                       256          95.05\% (0.23\%)  0.1774 (0.0049)   
MorpherNetv2 0         64           74.73\% (1.01\%)  1.0278 (0.0135)   
                       128          85.78\% (0.26\%)  0.5107 (0.0057)   
                       256          89.23\% (0.22\%)  0.3823 (0.0033)   
             1         64           85.92\% (0.25\%)  0.4665 (0.0050)   
                       128          92.91\% (0.08\%)  0.2400 (0.0038)   
                       256          94.67\% (0.24\%)  0.1808 (0.0089)   
             3         64           86.60\% (0.52\%)  0.4186 (0.0258)   
                       128          94.89\% (0.25\%)  0.1658 (0.0065)   
                       256          95.64\% (0.10\%)  0.1507 (0.0039)   

                                    \# parameters  
model        \# hidden hidden size                 
BaselineNet  0         64                   50890  
                       128                 101770  
                       256                 203530  
             1         64                   55050  
                       128                 118282  
                       256                 269322  
             3         64                   63370  
                       128                 151306  
                       256                 400906  
MorpherNet   0         64                    7818  
                       128                  18698  
                       256                  52746  
             1         64                   11978  
                       128                  35210  
                       256                 118538  
             3         64                   20298  
                       128                  68234  
                       256                 250122  
MorpherNetv2 0         64                    7178  
                       128                  17546  
                       256                  52746  
             1         64                   11338  
                       128                  34058  
                       256                 118538  
             3         64                   19658  
                       128                  67082  
                       256                 250122

In [43]:
print(aggregated.to_latex())

\begin{tabular}{lllllr}
\toprule
 &  &  & accuracy & CE & \# parameters \\
model & \# hidden & hidden size &  &  &  \\
\midrule
\multirow[t]{9}{*}{BaselineNet} & \multirow[t]{3}{*}{0} & 64 & 97.16\% (0.12\%) & 0.0975 (0.0018) & 50890 \\
 &  & 128 & 97.59\% (0.08\%) & 0.0830 (0.0027) & 101770 \\
 &  & 256 & 97.81\% (0.11\%) & 0.0759 (0.0031) & 203530 \\
\cline{2-6}
 & \multirow[t]{3}{*}{1} & 64 & 97.17\% (0.12\%) & 0.0993 (0.0027) & 55050 \\
 &  & 128 & 97.48\% (0.16\%) & 0.0869 (0.0056) & 118282 \\
 &  & 256 & 97.78\% (0.22\%) & 0.0824 (0.0064) & 269322 \\
\cline{2-6}
 & \multirow[t]{3}{*}{3} & 64 & 96.89\% (0.11\%) & 0.1093 (0.0021) & 63370 \\
 &  & 128 & 97.37\% (0.16\%) & 0.0917 (0.0038) & 151306 \\
 &  & 256 & 97.69\% (0.25\%) & 0.0858 (0.0078) & 400906 \\
\cline{1-6} \cline{2-6}
\multirow[t]{9}{*}{MorpherNet} & \multirow[t]{3}{*}{0} & 64 & 87.68\% (0.21\%) & 0.4359 (0.0031) & 7818 \\
 &  & 128 & 89.43\% (0.14\%) & 0.3743 (0.0045) & 18698 \\
 &  & 256 & 90.93\% (0.10\%) & 0.3229 (0